In [ ]:
#necessary imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle
import os
import json
from sklearn.model_selection import StratifiedKFold

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms
import torchvision.models as models

In [ ]:
PATH = '/media/popo/Elements/Datasets/Leaf-disease-classification/'

In [ ]:
label_dict = json.load(open(f'{PATH}label_num_to_disease_map.json'))
train_ids = pd.read_csv(f'{PATH}train.csv')

In [ ]:
train_ids.head()

In [ ]:
#plot the distribution of the labels
train_ids['label'].value_counts().plot(kind='bar');

### There is class imbalance in the dataset, so we need to take care of it. We can use the stratified sampling method to balance the dataset.

In [ ]:
#apply transformations to the images
transform = transforms.Compose([
                transforms.ToPILImage(),
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
            ])

In [ ]:
#image dataset
class LeafDataset(Dataset):
    def __init__(self, df, path, transform=None):
        self.df = df
        self.path = path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name, img_label = self.df.iloc[idx]
        img_label = torch.tensor(img_label)
        # print(img_label)
        # print(img_name)
        img_path = os.path.join(self.path, img_name)
        #print(img_path)
        img = plt.imread(img_path)
        if self.transform:
            img = self.transform(img)
        return img, img_label


In [ ]:
#create image dataset
img_path = f'{PATH}train_images/'
df = train_ids.copy()
train_dataset = LeafDataset(df, img_path, transform=transform)

In [ ]:
#show some images
def show_images(dataset):
    #subplot 8 images
    fig,ax = plt.subplots(2,4, figsize=(16,8))
    for i in range(2):
        for j in range(4):
            idx = i*4+j
            ax[i,j].imshow(dataset[idx][0].permute(1,2,0).detach().numpy())
            for key, value in label_dict.items():
                if int(key) == dataset[idx][1]:
                    ax[i,j].set_title(value)
                    break
            ax[i,j].axis('off')
    plt.show();

In [ ]:
show_images(train_dataset)

In [ ]:
#define the device to use
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
#load Resnet34
model = models.resnet34(pretrained=True)
#change last layer
num_ftrs = model.fc.in_features #number of input features to the last layer
#len(label_dict) = 5 -> 5 classes
model.fc = nn.Linear(in_features = num_ftrs, out_features = len(label_dict)) 
model = model.to(device)

In [ ]:
#loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
#define the training function
def train(model, train_loader, val_loader, criterion, optimizer, num_epochs):
    #train the model
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(train_loader):
            images = images.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            if (i+1) % 50 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')
    #evaluate the model
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy of the model on the {len(val_ds)} val_set: {100 * correct / total:.2f}%')
    

### We will use straticified sampling method and there will be n folds in total. Using each fold we will create a training and testing datasets and the data loaders,

In [ ]:
#data loader
batch_size = 128
n_splits = 5
num_epochs = 1
#stratified kfold cross validation
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
for fold, (train_index, val_index) in enumerate(skf.split(df['image_id'], df['label'])):
    train_ds = Subset(train_dataset, train_index)
    val_ds = Subset(train_dataset, val_index)
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=False)
    val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)   
    #train the model
    print(f'Fold {fold+1}')
    train(model, train_loader, val_loader, criterion, optimizer, num_epochs=num_epochs)


In [ ]:
#save model as pickle
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)